# Running a multi-member hydrological ensemble on the Raven Server

Here we use birdy's WPS client to launch the GR4JCN hydrological model using two sets of parameters.

In [1]:
from birdy import WPSClient

from example_data import TESTDATA
import datetime as dt
from urllib.request import urlretrieve
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt

# url = "https://pavics.ouranos.ca/twitcher/ows/proxy/raven/wps"
url = "http://localhost:9099/wps"
wps = WPSClient(url)

In [2]:
# The model parameters for gr4jcn and hmets. 
# It's not possible at the moment to pass multiple parameters as a nested list due to the WPSClient limitations. 
# Use a list of strings instead. 
params = ['0.529, -3.396, 407.29, 1.072, 16.9, 0.947', '0.4, -3.96, 307.29, 1.072, 16.9, 0.947']

# Forcing files. Raven uses the same forcing files for all and extracts the information it requires for each model.
ts=TESTDATA['raven-gr4j-cemaneige-nc-ts']

# Model configuration parameters.
config = dict(
    start_date=dt.datetime(2000, 1, 1),
    end_date=dt.datetime(2002, 1, 1),
    area=4250.6,
    name="Salmon",
    elevation=843.0,
    latitude=54.4848,
    longitude=-123.3659,
    )

# Launch the WPS to get the multi-model results.  Note the "gr4jcn" and "hmets" keys.
resp = wps.raven_gr4j_cemaneige(ts=str(ts), params=params, **config) 

The `hydrograph` and `storage` output are netCDF files where the simulations are concatenated along the `params` dimension. The `solution` output is a list of `rvc` files, and the `diagnostics` a list of performance metrics comparing simulations with observations. 

In [6]:
### asobj=True is failing!
[hydrograph, storage, solution, diagnostics] = resp.get(asobj=True)
hydrograph

AttributeError: 'str' object has no attribute 'identifier'

In [7]:
hydrograph.q_sim.isel(nbasins=0).plot.line(hue='params')

AttributeError: 'str' object has no attribute 'q_sim'

In [8]:
print(diagnostics)

http://localhost:9099/outputs/70a182c2-cf52-11e9-990b-484d7ef8d8d1/Diagnostics.zip
